# SQLModel

SQLModel is a library for interacting with SQL databases in Python. It is designed to be intuitive and easy to use. It is powered by [Pydantic](https://docs.pydantic.dev/latest/) and [SQLAlchemy](https://sqlalchemy.org/).

It combines SQLAlchemy and Pydantic and tries to simplify the code you write as much as possible, allowing you to reduce the code duplication to a minimum, but while getting the best developer experience possible.

In [1]:
from typing import Optional

from sqlmodel import Field, SQLModel

class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

In [2]:
hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
hero_2 = Hero(name="Spider", secret_name="Peter Parkour", age=18)
hero_3 = Hero(name="Rusty", secret_name="Tommy Sharp", age=48)

Writing to the database

In [3]:
import os
from sqlmodel import create_engine, Session

engine = create_engine("sqlite:///%s" % os.path.join(os.getcwd(), "example.sqlite3"))
SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    session.add(hero_1)
    session.add(hero_2)
    session.add(hero_3)
    session.commit()

Reading from the database

In [5]:
from sqlmodel import select

with Session(engine) as session:
    statement = select(Hero).where(Hero.name == "Spider")
    heros = session.exec(statement).all()
    print(f"Values: {hero}, type: {type(heros)}")

Values: [Hero(age=18, secret_name='Peter Parkour', name='Spider', id=2)], type: <class 'list'>
